In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\lucia\OneDrive\Documentos\GitHub\CD DP\P2\Projeto2CDDP\Projeto2CDDP-master


In [3]:
base_dados = pd.read_excel('spamhamV2.xlsx')

In [4]:
Treinamento = base_dados.sample(n=None, frac=0.75, replace=False).reset_index()
Teste = base_dados.sample(n=None, frac=0.25, replace=False).reset_index()

Treinamento = Treinamento.drop(["index"],axis = 1)
Teste = Teste.drop(["index"],axis = 1)

In [5]:
#######

In [6]:
n = base_dados.Email.count()

troca = ['÷','  ','‰','!','@','#','$','%','¨','&','*','(',')','-','_','+','=','§','{','}','[',']','ª','º','?','/','°','\n','|',',','.',';',':','å','£']

for e in range(n):
    base_dados.Email[e:e+1] = base_dados.Email[e:e+1].str.replace('  ',' ')
    for i in troca:
        base_dados.Email[e:e+1] = base_dados.Email[e:e+1].str.replace(i,'')

base_dados.head()

,Email,Class
0,Go until jurong point crazy Available only in ...,ham
1,Ok lar Joking wif u oni,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor U c already then say,ham
4,Nah I don't think he goes to usf he lives arou...,ham


In [7]:
EHAM= Treinamento.Email[Treinamento.Class=='ham'].values
ESPAM= Treinamento.Email[Treinamento.Class=='spam'].values

valuecounts_Treinamento = pd.DataFrame(Treinamento.Class.value_counts(True))
valuecounts_Treinamento

,Class
ham,0.864322
spam,0.135678


In [8]:
Eham = 0
Espam = 0 
Etotal = len(Treinamento)
for i in Treinamento['Class']:
    if(i == 'ham'):
        Eham += 1
    else:
        Espam+=1
        
print('De um total de {0} emails, {1} são considerados Ham e {2} são considerados Spam.'.format(Etotal,Eham,Espam))
Prob_h = Eham/Etotal
Prob_s = Espam/Etotal
print('Logo, a probabilidade de um email ser considerado Ham é de {0}%, e de ser Spam é de {1}%'.format(Prob_h*100,Prob_s*100))

De um total de 4179 emails, 3612 são considerados Ham e 567 são considerados Spam.
Logo, a probabilidade de um email ser considerado Ham é de 86.4321608040201%, e de ser Spam é de 13.5678391959799%


In [9]:
dici = {}
#dicti["palavra"]= ham,spam,pham,pspam
for i in range(Etotal):
    for palavra in Treinamento.Email[i]:
        if Treinamento.Class[i] == "ham":
            if palavra not in dici:
                dici[palavra] = 1,0
            else:
                h=dici[palavra][0]
                s=dici[palavra][1]
                dici[palavra]=h+1,s
        else:
            if palavra not in dici:
                dici[palavra] = 0,1
            else:
                h=dici[palavra][0]
                s=dici[palavra][1]
                dici[palavra]=h,s+1
                
Tham=0
Tspam=0
for i in dici:
    Tham+=dici[i][0]
    Tspam+=dici[i][1]
for i in dici:
    h=dici[i][0]
    s=dici[i][1]
    P_ham=(h+1)/((Tham*2)+Tspam)
    P_spam=(s+1)/((Tspam*2)+Tham)
    dici[i]=h,s,P_ham,P_spam



In [10]:
# palavras = {'palavras': palavras_total,'classificacao': classificacao}
# wordset = pd.DataFrame(data=palavras)
# wordset.head()

# i=1 
# while i < len(wordset):
#     for p in dici:
#         achou = False
#         if wordset['palavras'][i] == p[0]:
#             if wordset['classificacao'][i] == 'ham':
#                 p[1] += 1
#                 achou = True
#                 break
#             else:
#                 p[2] += 1
#                 achou = True
#                 break
#     if not achou:
#         if(wordset['classificacao'][i] == 'ham'):
#             dici.append([wordset['palavras'][i],1,0])
#         else:
#             dici.append([wordset['palavras'][i],0,1])
            
       
#     i+=1



In [11]:
#probabilidade de uma palavra ( ) acontecer se a mensagem na base treinamento é considerada SPAM
# Ham = 0
# Spam = 0 
# for i  in dici:
#     if h > 0:
#         Ham+=1
#     if s > 0:
#         Spam+=1
# print('A quantidade de palavras que podem ser encontradas em um email Ham é de {}, e a quantidade de palavras que podem ser encontradas em um email Spam é de {}.'.format(Ham,Spam))

In [12]:
def NB (frase):
    Prob_h = Eham/Etotal
    Prob_s = Espam/Etotal

    for palavra in frase:
        if palavra in dici:
            Prob_s = Prob_s* dici[palavra][3]
            Prob_h=Prob_h* dici[palavra][2]
           
            
        else:
            Prob_s = (Prob_s+1)/dici
            Prob_h = (Prob_h+1)/dici
    if Prob_s>Prob_h:
        return("spam")
    else:
        return("ham")
    
def funcao_test(Teste):
    frase = Teste.Email
    return NB(frase)

Test = Teste.apply(funcao_test,axis=1)
Teste["NB"] = Test

In [13]:
VN=Teste.loc[(Teste.Class == "ham") & (Teste.NB == "ham"),"result"] = "verdadeiro negativo"
VP=Teste.loc[(Teste.Class == "spam") & (Teste.NB == "spam"),"result"] = "verdadeiro positivo "
FP=Teste.loc[(Teste.Class == "spam") & (Teste.NB == "ham"),"result"] = "falso positivo"
FN=Teste.loc[(Teste.Class == "ham") & (Teste.NB == "sam"),"result"] = "falso negativo"
Teste.result.value_counts()

verdadeiro negativo     1200
falso positivo           189
verdadeiro positivo        4
Name: result, dtype: int64

In [14]:
rodrigo=pd.crosstab(Teste.Class, Teste.NB)
rodrigo

NB,ham,spam
Class,,
ham,1200,0
spam,189,4


In [15]:
VP=rodrigo.ham[0]
FP=rodrigo.ham[1]
FN=rodrigo.spam[0]
VN=rodrigo.spam[1]

In [16]:

P_VN = (VN/len(Teste))*100
P_VP = (VP/len(Teste))*100
P_FP = (FP/len(Teste))*100
P_FN = (FN/len(Teste))*100

print("A probabilidade de acontecer um verdadeiro negativo é de {0}%." .format(P_VN))
print("A probabilidade de acontecer um verdadeiro positivo é de {0}%." .format(P_VP))
print("A probabilidade de acontecer um falso positivo é de {0}%." .format(P_FP))
print("A probabilidade de acontecer um falso negativo é de {0}%." .format(P_FN))

A probabilidade de acontecer um verdadeiro negativo é de 0.2871500358937545%.
A probabilidade de acontecer um verdadeiro positivo é de 86.14501076812635%.
A probabilidade de acontecer um falso positivo é de 13.5678391959799%.
A probabilidade de acontecer um falso negativo é de 0.0%.


In [17]:
Teste.head()

,Email,Class,NB,result
0,The 2 oz guy is being kinda flaky but one frie...,ham,ham,verdadeiro negativo
1,K.then any other special?,ham,ham,verdadeiro negativo
2,Are you staying in town ?,ham,ham,verdadeiro negativo
3,Yup Ì_ not comin :-(,ham,ham,verdadeiro negativo
4,WOT U WANNA DO THEN MISSY?,ham,ham,verdadeiro negativo
